In [1]:
%%capture
import ROOT
import os

ROOT.gInterpreter.Declare('#include "postselection.h"')

Error in <TExMap::Add>: key 139970775489712 is not unique
Error in <TExMap::Add>: key 139970776014544 is not unique
In file included from input_line_44:1:
./postselection.h:712:1: warning: control may reach end of non-void function [-Wreturn-type]
}
^


In [2]:
#os.environ["X509_USER_PROXY"] = "/eos/home-t/ttedesch/SWAN_projects/RDataFrame_porting/proxy"
ROOT.gInterpreter.Declare(
"""
TMVA::Experimental::RBDT<> bdt_tagger("tagger", "https://ttedesch.web.cern.ch/ttedesch/VBSTagger_XGB.root");
TMVA::Experimental::RBDT<> bdt_tagger_mjj("tagger", "https://ttedesch.web.cern.ch/ttedesch/VBSTagger_XGB_mjj.root");
TMVA::Experimental::RBDT<> bdt_tagger_mjj_deltaetamjj("VBSTagger_XGB", "https://ttedesch.web.cern.ch/ttedesch/VBSTagger_XGB_mjj_absdeltaetajj.root");
TMVA::Experimental::RBDT<> bdt_tagger_mjj_deltaetamjj_new("VBSTagger_XGB", "https://ttedesch.web.cern.ch/ttedesch/VBSTagger_XGB_mjj_absdeltaetajj_new.root");
TMVA::Experimental::RBDT<> bdt_tagger_mjj_deltaetamjj_flavourmatching("VBSTagger_XGB", "https://ttedesch.web.cern.ch/ttedesch/VBSTagger_XGB_mjj_absdeltaetajj_flavourMatching.root");
TMVA::Experimental::RBDT<> bdt_tagger_mjj_deltaetamjj_flavourmatching_clean_fixed("VBSTagger_XGB", "https://ttedesch.web.cern.ch/ttedesch/VBSTagger_XGB_mjj_absdeltaetajj_flavourMatching_clean_fixed.root");
TMVA::Experimental::RBDT<> bdt_tagger_mjj_deltaetamjj_flavourmatching_clean_fixed_central("VBSTagger_XGB", "https://ttedesch.web.cern.ch/ttedesch/VBSTagger_XGB_mjj_absdeltaetajj_flavourMatching_clean_fixed_central.root");


RVec<size_t> SelectVBSJets_tagger(rvec_f pt, rvec_f eta, rvec_f phi, rvec_f mass, rvec_f Jet_area, rvec_f Jet_chHEF, rvec_f Jet_muEF, rvec_f Jet_neEmEF, rvec_f Jet_neHEF, rvec_f Jet_jetId, rvec_f Jet_nConstituents, rvec_f Jet_nElectrons, rvec_f Jet_nMuons, rvec_i GoodJets_idx)
{

    RVec<size_t> idx;
    // Find first lepton pair with invariant mass closest to Z mass
    auto idx_cmb = Combinations(GoodJets_idx, 2);
    float best_score = -1.;
    size_t best_i1 = 0; size_t best_i2 = 0;
    for (size_t i = 0; i < idx_cmb[0].size(); i++) {
        const auto i1 = idx_cmb[0][i];
        const auto i2 = idx_cmb[1][i];
        //std::cout<<GoodJets_idx[i1]<<GoodJets_idx[i2]<<endl;
        ROOT::Math::PtEtaPhiMVector p1(pt[GoodJets_idx[i1]], eta[GoodJets_idx[i1]], phi[GoodJets_idx[i1]], mass[GoodJets_idx[i1]]);
        ROOT::Math::PtEtaPhiMVector p2(pt[GoodJets_idx[i2]], eta[GoodJets_idx[i2]], phi[GoodJets_idx[i2]], mass[GoodJets_idx[i2]]);
        const auto this_mass = (p1 + p2).M();
        if (abs(eta[GoodJets_idx[i1]] - eta[GoodJets_idx[i2]]) >= DELTAETA_JJ_CUT && (p1 + p2).M() > 500) {
            auto y = bdt_tagger.Compute({Jet_area[GoodJets_idx[i1]], Jet_chHEF[GoodJets_idx[i1]], Jet_muEF[GoodJets_idx[i1]], Jet_neEmEF[GoodJets_idx[i1]], Jet_neHEF[GoodJets_idx[i1]], Jet_jetId[GoodJets_idx[i1]], Jet_nConstituents[GoodJets_idx[i1]], Jet_nElectrons[GoodJets_idx[i1]], Jet_nMuons[GoodJets_idx[i1]], Jet_area[GoodJets_idx[i2]], Jet_chHEF[GoodJets_idx[i2]], Jet_muEF[GoodJets_idx[i2]], Jet_neEmEF[GoodJets_idx[i2]], Jet_neHEF[GoodJets_idx[i2]], Jet_jetId[GoodJets_idx[i2]], Jet_nConstituents[GoodJets_idx[i2]], Jet_nElectrons[GoodJets_idx[i2]], Jet_nMuons[GoodJets_idx[i2]]});
            float this_score = y[0];
            //cout<<"this score: "<<this_score<<endl;
            //cout<<"best score: "<<best_score<<endl;
            //cout<<endl;
            if (this_score > best_score) {
                best_score = this_score;
                best_i1 = GoodJets_idx[i1];
                best_i2 = GoodJets_idx[i2];
            }
        }
    } 
    //cout<<"best score of the event: "<<best_score<<endl;
    //cout<<endl;
    //cout<<endl;
    idx.emplace_back(best_i1);
    idx.emplace_back(best_i2);
    return idx;
}

RVec<size_t> SelectVBSJets_tagger_mjj(rvec_f pt, rvec_f eta, rvec_f phi, rvec_f mass, rvec_f Jet_area, rvec_f Jet_chHEF, rvec_f Jet_muEF, rvec_f Jet_neEmEF, rvec_f Jet_neHEF, rvec_f Jet_jetId, rvec_f Jet_nConstituents, rvec_f Jet_nElectrons, rvec_f Jet_nMuons, rvec_i GoodJets_idx)
{

    RVec<size_t> idx;
    // Find first lepton pair with invariant mass closest to Z mass
    auto idx_cmb = Combinations(GoodJets_idx, 2);
    float best_score = -1.;
    size_t best_i1 = 0; size_t best_i2 = 0;
    for (size_t i = 0; i < idx_cmb[0].size(); i++) {
        const auto i1 = idx_cmb[0][i];
        const auto i2 = idx_cmb[1][i];
        //std::cout<<GoodJets_idx[i1]<<GoodJets_idx[i2]<<endl;
        ROOT::Math::PtEtaPhiMVector p1(pt[GoodJets_idx[i1]], eta[GoodJets_idx[i1]], phi[GoodJets_idx[i1]], mass[GoodJets_idx[i1]]);
        ROOT::Math::PtEtaPhiMVector p2(pt[GoodJets_idx[i2]], eta[GoodJets_idx[i2]], phi[GoodJets_idx[i2]], mass[GoodJets_idx[i2]]);
        const float this_mass = (p1 + p2).M();
        if (abs(eta[GoodJets_idx[i1]] - eta[GoodJets_idx[i2]]) >= DELTAETA_JJ_CUT && (p1 + p2).M() > 500) {
            auto y = bdt_tagger_mjj.Compute({Jet_area[GoodJets_idx[i1]], Jet_chHEF[GoodJets_idx[i1]], Jet_muEF[GoodJets_idx[i1]], Jet_neEmEF[GoodJets_idx[i1]], Jet_neHEF[GoodJets_idx[i1]], Jet_jetId[GoodJets_idx[i1]], Jet_nConstituents[GoodJets_idx[i1]], Jet_nElectrons[GoodJets_idx[i1]], Jet_nMuons[GoodJets_idx[i1]], Jet_area[GoodJets_idx[i2]], Jet_chHEF[GoodJets_idx[i2]], Jet_muEF[GoodJets_idx[i2]], Jet_neEmEF[GoodJets_idx[i2]], Jet_neHEF[GoodJets_idx[i2]], Jet_jetId[GoodJets_idx[i2]], Jet_nConstituents[GoodJets_idx[i2]], Jet_nElectrons[GoodJets_idx[i2]], Jet_nMuons[GoodJets_idx[i2]], this_mass});
            float this_score = y[0];
            //cout<<"this score: "<<this_score<<endl;
            //cout<<"best score: "<<best_score<<endl;
            //cout<<endl;
            if (this_score > best_score) {
                best_score = this_score;
                best_i1 = GoodJets_idx[i1];
                best_i2 = GoodJets_idx[i2];
            }
        }
    } 
    //cout<<"best score of the event: "<<best_score<<endl;
    //cout<<endl;
    //cout<<endl;
    idx.emplace_back(best_i1);
    idx.emplace_back(best_i2);
    return idx;
}

RVec<size_t> SelectVBSJets_tagger_mjj_absdeltaetajj(rvec_f pt, rvec_f eta, rvec_f phi, rvec_f mass, rvec_f Jet_area, rvec_f Jet_chHEF, rvec_f Jet_muEF, rvec_f Jet_neEmEF, rvec_f Jet_neHEF, rvec_f Jet_jetId, rvec_f Jet_nConstituents, rvec_f Jet_nElectrons, rvec_f Jet_nMuons, rvec_i GoodJets_idx)
{

    RVec<size_t> idx;
    // Find first lepton pair with invariant mass closest to Z mass
    auto idx_cmb = Combinations(GoodJets_idx, 2);
    float best_score = -1.;
    size_t best_i1 = 0; size_t best_i2 = 0;
    for (size_t i = 0; i < idx_cmb[0].size(); i++) {
        const auto i1 = idx_cmb[0][i];
        const auto i2 = idx_cmb[1][i];
        //std::cout<<GoodJets_idx[i1]<<GoodJets_idx[i2]<<endl;
        ROOT::Math::PtEtaPhiMVector p1(pt[GoodJets_idx[i1]], eta[GoodJets_idx[i1]], phi[GoodJets_idx[i1]], mass[GoodJets_idx[i1]]);
        ROOT::Math::PtEtaPhiMVector p2(pt[GoodJets_idx[i2]], eta[GoodJets_idx[i2]], phi[GoodJets_idx[i2]], mass[GoodJets_idx[i2]]);
        const float this_mass = (p1 + p2).M();
        if (abs(eta[GoodJets_idx[i1]] - eta[GoodJets_idx[i2]]) >= DELTAETA_JJ_CUT && (p1 + p2).M() > 500) {
            auto y = bdt_tagger_mjj_deltaetamjj.Compute({pt[GoodJets_idx[i1]], eta[GoodJets_idx[i1]], phi[GoodJets_idx[i1]], mass[GoodJets_idx[i1]], Jet_area[GoodJets_idx[i1]], Jet_chHEF[GoodJets_idx[i1]], Jet_muEF[GoodJets_idx[i1]], Jet_neEmEF[GoodJets_idx[i1]], Jet_neHEF[GoodJets_idx[i1]], Jet_jetId[GoodJets_idx[i1]], Jet_nConstituents[GoodJets_idx[i1]], Jet_nElectrons[GoodJets_idx[i1]], Jet_nMuons[GoodJets_idx[i1]], pt[GoodJets_idx[i2]], eta[GoodJets_idx[i2]], phi[GoodJets_idx[i2]], mass[GoodJets_idx[i2]], Jet_area[GoodJets_idx[i2]], Jet_chHEF[GoodJets_idx[i2]], Jet_muEF[GoodJets_idx[i2]], Jet_neEmEF[GoodJets_idx[i2]], Jet_neHEF[GoodJets_idx[i2]], Jet_jetId[GoodJets_idx[i2]], Jet_nConstituents[GoodJets_idx[i2]], Jet_nElectrons[GoodJets_idx[i2]], Jet_nMuons[GoodJets_idx[i2]], abs(eta[GoodJets_idx[i1]] - eta[GoodJets_idx[i2]]),  this_mass});
            float this_score = y[0];
            //cout<<"this score: "<<this_score<<endl;
            //cout<<"best score: "<<best_score<<endl;
            //cout<<endl;
            if (this_score > best_score) {
                best_score = this_score;
                best_i1 = GoodJets_idx[i1];
                best_i2 = GoodJets_idx[i2];
            }
        }
    } 
    //cout<<"best score of the event: "<<best_score<<endl;
    //cout<<endl;
    //cout<<endl;
    idx.emplace_back(best_i1);
    idx.emplace_back(best_i2);
    return idx;
}

RVec<size_t> SelectVBSJets_tagger_mjj_absdeltaetajj_new(rvec_f pt, rvec_f eta, rvec_f phi, rvec_f mass, rvec_f Jet_area, rvec_f Jet_chHEF, rvec_f Jet_muEF, rvec_f Jet_neEmEF, rvec_f Jet_neHEF, rvec_f Jet_jetId, rvec_f Jet_nConstituents, rvec_f Jet_nElectrons, rvec_f Jet_nMuons, rvec_i GoodJets_idx)
{

    RVec<size_t> idx;
    // Find first lepton pair with invariant mass closest to Z mass
    auto idx_cmb = Combinations(GoodJets_idx, 2);
    float best_score = -1.;
    size_t best_i1 = 0; size_t best_i2 = 0;
    for (size_t i = 0; i < idx_cmb[0].size(); i++) {
        const auto i1 = idx_cmb[0][i];
        const auto i2 = idx_cmb[1][i];
        //std::cout<<GoodJets_idx[i1]<<GoodJets_idx[i2]<<endl;
        ROOT::Math::PtEtaPhiMVector p1(pt[GoodJets_idx[i1]], eta[GoodJets_idx[i1]], phi[GoodJets_idx[i1]], mass[GoodJets_idx[i1]]);
        ROOT::Math::PtEtaPhiMVector p2(pt[GoodJets_idx[i2]], eta[GoodJets_idx[i2]], phi[GoodJets_idx[i2]], mass[GoodJets_idx[i2]]);
        const float this_mass = (p1 + p2).M();
        if (abs(eta[GoodJets_idx[i1]] - eta[GoodJets_idx[i2]]) >= DELTAETA_JJ_CUT && (p1 + p2).M() > 500) {
            auto y = bdt_tagger_mjj_deltaetamjj_new.Compute({pt[GoodJets_idx[i1]], eta[GoodJets_idx[i1]], phi[GoodJets_idx[i1]], mass[GoodJets_idx[i1]], Jet_area[GoodJets_idx[i1]], Jet_chHEF[GoodJets_idx[i1]], Jet_muEF[GoodJets_idx[i1]], Jet_neEmEF[GoodJets_idx[i1]], Jet_neHEF[GoodJets_idx[i1]], Jet_jetId[GoodJets_idx[i1]], Jet_nConstituents[GoodJets_idx[i1]], Jet_nElectrons[GoodJets_idx[i1]], Jet_nMuons[GoodJets_idx[i1]], pt[GoodJets_idx[i2]], eta[GoodJets_idx[i2]], phi[GoodJets_idx[i2]], mass[GoodJets_idx[i2]], Jet_area[GoodJets_idx[i2]], Jet_chHEF[GoodJets_idx[i2]], Jet_muEF[GoodJets_idx[i2]], Jet_neEmEF[GoodJets_idx[i2]], Jet_neHEF[GoodJets_idx[i2]], Jet_jetId[GoodJets_idx[i2]], Jet_nConstituents[GoodJets_idx[i2]], Jet_nElectrons[GoodJets_idx[i2]], Jet_nMuons[GoodJets_idx[i2]], abs(eta[GoodJets_idx[i1]] - eta[GoodJets_idx[i2]]),  this_mass});
            float this_score = y[0];
            //cout<<"this score: "<<this_score<<endl;
            //cout<<"best score: "<<best_score<<endl;
            //cout<<endl;
            if (this_score > best_score) {
                best_score = this_score;
                best_i1 = GoodJets_idx[i1];
                best_i2 = GoodJets_idx[i2];
            }
        }
    } 
    //cout<<"best score of the event: "<<best_score<<endl;
    //cout<<endl;
    //cout<<endl;
    idx.emplace_back(best_i1);
    idx.emplace_back(best_i2);
    return idx;
}

RVec<size_t> SelectVBSJets_tagger_mjj_absdeltaetajj_flavourmatching(rvec_f pt, rvec_f eta, rvec_f phi, rvec_f mass, rvec_f Jet_area, rvec_f Jet_chHEF, rvec_f Jet_muEF, rvec_f Jet_neEmEF, rvec_f Jet_neHEF, rvec_f Jet_jetId, rvec_f Jet_nConstituents, rvec_f Jet_nElectrons, rvec_f Jet_nMuons, rvec_i GoodJets_idx)
{

    RVec<size_t> idx;
    // Find first lepton pair with invariant mass closest to Z mass
    auto idx_cmb = Combinations(GoodJets_idx, 2);
    float best_score = -1.;
    size_t best_i1 = 0; size_t best_i2 = 0;
    for (size_t i = 0; i < idx_cmb[0].size(); i++) {
        const auto i1 = idx_cmb[0][i];
        const auto i2 = idx_cmb[1][i];
        //std::cout<<GoodJets_idx[i1]<<GoodJets_idx[i2]<<endl;
        ROOT::Math::PtEtaPhiMVector p1(pt[GoodJets_idx[i1]], eta[GoodJets_idx[i1]], phi[GoodJets_idx[i1]], mass[GoodJets_idx[i1]]);
        ROOT::Math::PtEtaPhiMVector p2(pt[GoodJets_idx[i2]], eta[GoodJets_idx[i2]], phi[GoodJets_idx[i2]], mass[GoodJets_idx[i2]]);
        const float this_mass = (p1 + p2).M();
        if (abs(eta[GoodJets_idx[i1]] - eta[GoodJets_idx[i2]]) >= DELTAETA_JJ_CUT && (p1 + p2).M() > 500) {
            auto y = bdt_tagger_mjj_deltaetamjj_flavourmatching.Compute({pt[GoodJets_idx[i1]], eta[GoodJets_idx[i1]], phi[GoodJets_idx[i1]], mass[GoodJets_idx[i1]], Jet_area[GoodJets_idx[i1]], Jet_chHEF[GoodJets_idx[i1]], Jet_muEF[GoodJets_idx[i1]], Jet_neEmEF[GoodJets_idx[i1]], Jet_neHEF[GoodJets_idx[i1]], Jet_jetId[GoodJets_idx[i1]], Jet_nConstituents[GoodJets_idx[i1]], Jet_nElectrons[GoodJets_idx[i1]], Jet_nMuons[GoodJets_idx[i1]], pt[GoodJets_idx[i2]], eta[GoodJets_idx[i2]], phi[GoodJets_idx[i2]], mass[GoodJets_idx[i2]], Jet_area[GoodJets_idx[i2]], Jet_chHEF[GoodJets_idx[i2]], Jet_muEF[GoodJets_idx[i2]], Jet_neEmEF[GoodJets_idx[i2]], Jet_neHEF[GoodJets_idx[i2]], Jet_jetId[GoodJets_idx[i2]], Jet_nConstituents[GoodJets_idx[i2]], Jet_nElectrons[GoodJets_idx[i2]], Jet_nMuons[GoodJets_idx[i2]], abs(eta[GoodJets_idx[i1]] - eta[GoodJets_idx[i2]]),  this_mass});
            float this_score = y[0];
            //cout<<"this score: "<<this_score<<endl;
            //cout<<"best score: "<<best_score<<endl;
            //cout<<endl;
            if (this_score > best_score) {
                best_score = this_score;
                best_i1 = GoodJets_idx[i1];
                best_i2 = GoodJets_idx[i2];
            }
        }
    } 
    //cout<<"best score of the event: "<<best_score<<endl;
    //cout<<endl;
    //cout<<endl;
    idx.emplace_back(best_i1);
    idx.emplace_back(best_i2);
    return idx;
}

RVec<size_t> SelectVBSJets_tagger_mjj_absdeltaetajj_flavourmatching_clean_fixed(rvec_f pt, rvec_f eta, rvec_f phi, rvec_f mass, rvec_f Jet_area, rvec_f Jet_chHEF, rvec_f Jet_muEF, rvec_f Jet_neEmEF, rvec_f Jet_neHEF, rvec_f Jet_jetId, rvec_f Jet_nConstituents, rvec_f Jet_nElectrons, rvec_f Jet_nMuons, rvec_i GoodJets_idx)
{

    RVec<size_t> idx;
    // Find first lepton pair with invariant mass closest to Z mass
    auto idx_cmb = Combinations(GoodJets_idx, 2);
    float best_score = -1.;
    size_t best_i1 = 0; size_t best_i2 = 0;
    for (size_t i = 0; i < idx_cmb[0].size(); i++) {
        const auto i1 = idx_cmb[0][i];
        const auto i2 = idx_cmb[1][i];
        //std::cout<<GoodJets_idx[i1]<<GoodJets_idx[i2]<<endl;
        ROOT::Math::PtEtaPhiMVector p1(pt[GoodJets_idx[i1]], eta[GoodJets_idx[i1]], phi[GoodJets_idx[i1]], mass[GoodJets_idx[i1]]);
        ROOT::Math::PtEtaPhiMVector p2(pt[GoodJets_idx[i2]], eta[GoodJets_idx[i2]], phi[GoodJets_idx[i2]], mass[GoodJets_idx[i2]]);
        const float this_mass = (p1 + p2).M();
        if (abs(eta[GoodJets_idx[i1]] - eta[GoodJets_idx[i2]]) >= DELTAETA_JJ_CUT && (p1 + p2).M() > 500) {
            auto y = bdt_tagger_mjj_deltaetamjj_flavourmatching_clean_fixed.Compute({pt[GoodJets_idx[i1]], eta[GoodJets_idx[i1]], phi[GoodJets_idx[i1]], mass[GoodJets_idx[i1]], Jet_area[GoodJets_idx[i1]], Jet_chHEF[GoodJets_idx[i1]], Jet_muEF[GoodJets_idx[i1]], Jet_neEmEF[GoodJets_idx[i1]], Jet_neHEF[GoodJets_idx[i1]], Jet_jetId[GoodJets_idx[i1]], Jet_nConstituents[GoodJets_idx[i1]], Jet_nElectrons[GoodJets_idx[i1]], Jet_nMuons[GoodJets_idx[i1]], pt[GoodJets_idx[i2]], eta[GoodJets_idx[i2]], phi[GoodJets_idx[i2]], mass[GoodJets_idx[i2]], Jet_area[GoodJets_idx[i2]], Jet_chHEF[GoodJets_idx[i2]], Jet_muEF[GoodJets_idx[i2]], Jet_neEmEF[GoodJets_idx[i2]], Jet_neHEF[GoodJets_idx[i2]], Jet_jetId[GoodJets_idx[i2]], Jet_nConstituents[GoodJets_idx[i2]], Jet_nElectrons[GoodJets_idx[i2]], Jet_nMuons[GoodJets_idx[i2]], abs(eta[GoodJets_idx[i1]] - eta[GoodJets_idx[i2]]),  this_mass});
            float this_score = y[0];
            //cout<<"this score: "<<this_score<<endl;
            //cout<<"best score: "<<best_score<<endl;
            //cout<<endl;
            if (this_score > best_score) {
                best_score = this_score;
                best_i1 = GoodJets_idx[i1];
                best_i2 = GoodJets_idx[i2];
            }
        }
    } 
    //cout<<"best score of the event: "<<best_score<<endl;
    //cout<<endl;
    //cout<<endl;
    idx.emplace_back(best_i1);
    idx.emplace_back(best_i2);
    return idx;
}


/*
R__LOAD_LIBRARY(/usr/local/lib/libtensorflow.so)
#include "cppflow/ops.h"
#include "cppflow/model.h"
cppflow::model VBSTagger_DNN_model("./dnn_optimized_model_tagger_mjjabsdeltaetajj");
// C Library for dynamic linking
#include  <dlfcn.h>
// Define the type for generic machine learning functions
typedef float *(*mlfunc)(float *, const float*);



//float DNNinference(float x1, float x2, float x3, float x4, float x5, float x6, float x7, float x8, float x9, float x10, float x11, float x12, float x13, float x14, float x15, float x16, float x17, float x18, float x19, float x20, float x21, float x22, float x23, float x24, float x25, float x26, float x27, float x28){
//    auto input = cppflow::fill({1, 28}, 1.0f);
//    auto output = model(input);
//    auto values = output.get_data<float>();
//    return values[0];
//}


RVec<size_t> SelectVBSJets_tagger_mjj_absdeltaetajj_DNN(rvec_f pt, rvec_f eta, rvec_f phi, rvec_f mass, rvec_f Jet_area, rvec_f Jet_chHEF, rvec_f Jet_muEF, rvec_f Jet_neEmEF, rvec_f Jet_neHEF, rvec_f Jet_jetId, rvec_f Jet_nConstituents, rvec_f Jet_nElectrons, rvec_f Jet_nMuons, rvec_i GoodJets_idx)
{

    RVec<size_t> idx;
    // Find first lepton pair with invariant mass closest to Z mass
    auto idx_cmb = Combinations(GoodJets_idx, 2);
    float best_score = -1.;
    size_t best_i1 = 0; size_t best_i2 = 0;
    for (size_t i = 0; i < idx_cmb[0].size(); i++) {
        const auto i1 = idx_cmb[0][i];
        const auto i2 = idx_cmb[1][i];
        //std::cout<<GoodJets_idx[i1]<<GoodJets_idx[i2]<<endl;
        ROOT::Math::PtEtaPhiMVector p1(pt[GoodJets_idx[i1]], eta[GoodJets_idx[i1]], phi[GoodJets_idx[i1]], mass[GoodJets_idx[i1]]);
        ROOT::Math::PtEtaPhiMVector p2(pt[GoodJets_idx[i2]], eta[GoodJets_idx[i2]], phi[GoodJets_idx[i2]], mass[GoodJets_idx[i2]]);
        const float this_mass = (p1 + p2).M();
        if (abs(eta[GoodJets_idx[i1]] - eta[GoodJets_idx[i2]]) >= DELTAETA_JJ_CUT && (p1 + p2).M() > 500) {
            void *handle = dlopen ( "./deployed_scaler_tagger_mjjabsdeltaetajj.so", RTLD_LAZY );
            if (!handle) exit(1);
            mlfunc minmax_ = mlfunc(dlsym (handle, "MinMaxScaler_tagger_mjjabsdeltaetajj")); 
            float input_not_scaled [] = {pt[GoodJets_idx[i1]], eta[GoodJets_idx[i1]], phi[GoodJets_idx[i1]], mass[GoodJets_idx[i1]], Jet_area[GoodJets_idx[i1]], Jet_chHEF[GoodJets_idx[i1]], Jet_muEF[GoodJets_idx[i1]], Jet_neEmEF[GoodJets_idx[i1]], Jet_neHEF[GoodJets_idx[i1]], Jet_jetId[GoodJets_idx[i1]], Jet_nConstituents[GoodJets_idx[i1]], Jet_nElectrons[GoodJets_idx[i1]], Jet_nMuons[GoodJets_idx[i1]], pt[GoodJets_idx[i2]], eta[GoodJets_idx[i2]], phi[GoodJets_idx[i2]], mass[GoodJets_idx[i2]], Jet_area[GoodJets_idx[i2]], Jet_chHEF[GoodJets_idx[i2]], Jet_muEF[GoodJets_idx[i2]], Jet_neEmEF[GoodJets_idx[i2]], Jet_neHEF[GoodJets_idx[i2]], Jet_jetId[GoodJets_idx[i2]], Jet_nConstituents[GoodJets_idx[i2]], Jet_nElectrons[GoodJets_idx[i2]], Jet_nMuons[GoodJets_idx[i2]], abs(eta[GoodJets_idx[i1]] - eta[GoodJets_idx[i2]]),  this_mass};
            float input_scaled [28];
            minmax_(input_scaled, input_not_scaled);            
            auto input_scaled_list = {input_scaled[0],input_scaled[1],input_scaled[2],input_scaled[3],input_scaled[4],input_scaled[5],input_scaled[6],input_scaled[7],input_scaled[8],input_scaled[9],input_scaled[10],input_scaled[11],input_scaled[12],input_scaled[13],input_scaled[14],input_scaled[15],input_scaled[16],input_scaled[17],input_scaled[18],input_scaled[19],input_scaled[20],input_scaled[21],input_scaled[22],input_scaled[23],input_scaled[24],input_scaled[25], input_scaled[26],input_scaled[27]};
            auto input = cppflow::tensor(input_scaled_list);
            input = cppflow::expand_dims(input, 0);
            auto output = VBSTagger_DNN_model(input);
            auto values = output.get_data<float>();
            float this_score = values[0];
            //cout<<"this score: "<<this_score<<endl;
            //cout<<"best score: "<<best_score<<endl;
            //cout<<endl;
            if (this_score > best_score) {
                best_score = this_score;
                best_i1 = GoodJets_idx[i1];
                best_i2 = GoodJets_idx[i2];
            }
        }
    } 
    //cout<<"best score of the event: "<<best_score<<endl;
    //cout<<endl;
    //cout<<endl;
    idx.emplace_back(best_i1);
    idx.emplace_back(best_i2);
    return idx;
}
*/

RVec<int> SelectVBSQGenJet_FlavourMatching(rvec_i GenPart_pdgId, rvec_i GenPart_genPartIdxMother, rvec_f GenPart_pt, rvec_f GenPart_eta, rvec_i GenJet_partonFlavour, rvec_f GenJet_pt, rvec_f GenJet_eta){

    RVec<int> GenPart_idx;
    
    for (int i = 0; i < GenPart_pdgId.size(); i++) {
        if(GenPart_genPartIdxMother[i]==0 && abs(GenPart_pdgId[i])>0 && abs(GenPart_pdgId[i])<10) GenPart_idx.emplace_back(i);
    }
    
    RVec<int> dummy_idx;
    dummy_idx.emplace_back(-9999);
    dummy_idx.emplace_back(-9999);
    
    if (GenPart_idx.size() != 2) return dummy_idx;
    
    int GenPart_idx1 = GenPart_idx[0];
    int GenPart_idx2 = GenPart_idx[1];
    
    if(GenPart_pt[GenPart_idx1] == GenPart_pt[GenPart_idx2] && GenPart_eta[GenPart_idx1] == GenPart_eta[GenPart_idx2]) return dummy_idx;
    
    int qflav1 = GenPart_pdgId[GenPart_idx1];
    int qflav2 = GenPart_pdgId[GenPart_idx2];
    
    RVec<int> LightGenJet_idx;

    for (int i = 0; i < GenJet_partonFlavour.size(); i++) {
        if(abs(GenJet_partonFlavour[i])>0 && abs(GenJet_partonFlavour[i])<10 && (GenJet_partonFlavour[i]==qflav1 || GenJet_partonFlavour[i]==qflav2)) LightGenJet_idx.emplace_back(i);
    }
    
    if(LightGenJet_idx.size() < 2) return dummy_idx;
    
    float discrim1 = 1000000.;
    float discrim2 = 1000000.;
    int idx_genjet1 = -1;
    int idx_genjet2 = -1;
    float tmpdiscr1;
    float tmpdiscr2;
           
    for (int i = 0; i < LightGenJet_idx.size(); i++) {
        tmpdiscr1 = abs(GenJet_eta[LightGenJet_idx[i]] -  GenPart_eta[GenPart_idx1]) + abs(GenJet_pt[LightGenJet_idx[i]] -  GenPart_pt[GenPart_idx1]);
        tmpdiscr2 = abs(GenJet_eta[LightGenJet_idx[i]] -  GenPart_eta[GenPart_idx2]) + abs(GenJet_pt[LightGenJet_idx[i]] -  GenPart_pt[GenPart_idx2]);
        if(tmpdiscr1 < discrim1 && GenJet_partonFlavour[LightGenJet_idx[i]]==qflav1){
            discrim1 = tmpdiscr1;
            idx_genjet1 = LightGenJet_idx[i];
        }
        if(tmpdiscr2 < discrim2 && GenJet_partonFlavour[LightGenJet_idx[i]]==qflav2){
            discrim2 = tmpdiscr2;
            idx_genjet2 = LightGenJet_idx[i];
        }
    }
    
    //cout<<idx_genjet1<<": "<<discrim1<<", "<<idx_genjet2<<": "<<discrim2<<endl;
    
    if((idx_genjet1 == -1 || idx_genjet2 == -1) || (idx_genjet1 == idx_genjet2)) return dummy_idx;
    if(discrim1 > 10. || discrim2 > 10.) return dummy_idx;
    
    RVec<int> finalgenjets_idx(2);
           
    if(GenJet_pt[idx_genjet1] > GenJet_pt[idx_genjet2]){ 
           finalgenjets_idx[0] = idx_genjet1;
           finalgenjets_idx[1] = idx_genjet2;
    }
    else{ 
           finalgenjets_idx[0] = idx_genjet2;
           finalgenjets_idx[1] = idx_genjet1;
    }
    return finalgenjets_idx;
}



""")


True

In [3]:
ROOT.gInterpreter.Declare(
"""
bool isCentral(float obj_eta, float obj_phi, float j1_eta, float j1_phi, float j2_eta, float j2_phi){
    float dRjj = deltaR(j1_eta, j1_phi, j2_eta, j2_phi);
    float dRoj1 = deltaR(obj_eta, obj_phi, j1_eta, j1_phi);
    float dRoj2 = deltaR(obj_eta, obj_phi, j2_eta, j2_phi);
    if(dRoj1 < dRjj && dRoj2 < dRjj) return true;
    else return false;
}

float centralActivity(rvec_f obj_pt_list, rvec_f obj_eta_list, rvec_f  obj_phi_list, float j1_eta, float j1_phi, float j2_eta, float j2_phi){
    float total_pT = 0;
    for(int i=0; i<obj_pt_list.size(); i++){
        if (isCentral(obj_eta_list[i], obj_phi_list[i], j1_eta, j1_phi, j2_eta, j2_phi)==true) total_pT = total_pT + obj_pt_list[i];
    }
    return total_pT;
}

RVec<size_t> SelectVBSJets_tagger_mjj_absdeltaetajj_flavourmatching_clean_fixed_central(rvec_f pt, rvec_f eta, rvec_f phi, rvec_f mass, rvec_f Jet_area, rvec_f Jet_chHEF, rvec_f Jet_muEF, rvec_f Jet_neEmEF, rvec_f Jet_neHEF, rvec_f Jet_jetId, rvec_f Jet_nConstituents, rvec_f Jet_nElectrons, rvec_f Jet_nMuons, rvec_i GoodJets_idx,  rvec_f Electron_pt, rvec_f Electron_eta, rvec_f Electron_phi, rvec_f Muon_pt, rvec_f Muon_eta, rvec_f Muon_phi)
{
    RVec<size_t> idx;
    // Find first lepton pair with invariant mass closest to Z mass
    auto idx_cmb = Combinations(GoodJets_idx, 2);
    float best_score = -1.;
    size_t best_i1 = 0; size_t best_i2 = 0;
    for (size_t i = 0; i < idx_cmb[0].size(); i++) {
        const auto i1 = idx_cmb[0][i];
        const auto i2 = idx_cmb[1][i];
        //std::cout<<GoodJets_idx[i1]<<GoodJets_idx[i2]<<endl;
        ROOT::Math::PtEtaPhiMVector p1(pt[GoodJets_idx[i1]], eta[GoodJets_idx[i1]], phi[GoodJets_idx[i1]], mass[GoodJets_idx[i1]]);
        ROOT::Math::PtEtaPhiMVector p2(pt[GoodJets_idx[i2]], eta[GoodJets_idx[i2]], phi[GoodJets_idx[i2]], mass[GoodJets_idx[i2]]);
        const float this_mass = (p1 + p2).M();
        if (abs(eta[GoodJets_idx[i1]] - eta[GoodJets_idx[i2]]) >= DELTAETA_JJ_CUT && (p1 + p2).M() > 500) {
            float centralHT = centralActivity(pt, eta, phi, eta[GoodJets_idx[i1]], phi[GoodJets_idx[i1]], eta[GoodJets_idx[i2]], phi[GoodJets_idx[i2]]);
            float centralET = centralActivity(Electron_pt, Electron_eta, Electron_phi, eta[GoodJets_idx[i1]], phi[GoodJets_idx[i1]], eta[GoodJets_idx[i2]], phi[GoodJets_idx[i2]]);
            float centralMuT = centralActivity(Muon_pt, Muon_eta, Muon_phi, eta[GoodJets_idx[i1]], phi[GoodJets_idx[i1]], eta[GoodJets_idx[i2]], phi[GoodJets_idx[i2]]);
            auto y = bdt_tagger_mjj_deltaetamjj_flavourmatching_clean_fixed_central.Compute({pt[GoodJets_idx[i1]], eta[GoodJets_idx[i1]], phi[GoodJets_idx[i1]], mass[GoodJets_idx[i1]], Jet_area[GoodJets_idx[i1]], Jet_chHEF[GoodJets_idx[i1]], Jet_muEF[GoodJets_idx[i1]], Jet_neEmEF[GoodJets_idx[i1]], Jet_neHEF[GoodJets_idx[i1]], Jet_jetId[GoodJets_idx[i1]], Jet_nConstituents[GoodJets_idx[i1]], Jet_nElectrons[GoodJets_idx[i1]], Jet_nMuons[GoodJets_idx[i1]], pt[GoodJets_idx[i2]], eta[GoodJets_idx[i2]], phi[GoodJets_idx[i2]], mass[GoodJets_idx[i2]], Jet_area[GoodJets_idx[i2]], Jet_chHEF[GoodJets_idx[i2]], Jet_muEF[GoodJets_idx[i2]], Jet_neEmEF[GoodJets_idx[i2]], Jet_neHEF[GoodJets_idx[i2]], Jet_jetId[GoodJets_idx[i2]], Jet_nConstituents[GoodJets_idx[i2]], Jet_nElectrons[GoodJets_idx[i2]], Jet_nMuons[GoodJets_idx[i2]], abs(eta[GoodJets_idx[i1]] - eta[GoodJets_idx[i2]]),  this_mass, centralHT, centralET, centralMuT});
            float this_score = y[0];
            //cout<<"this score: "<<this_score<<endl;
            //cout<<"best score: "<<best_score<<endl;
            //cout<<endl;
            if (this_score > best_score) {
                best_score = this_score;
                best_i1 = GoodJets_idx[i1];
                best_i2 = GoodJets_idx[i2];
            }
        }
    } 
    //cout<<"best score of the event: "<<best_score<<endl;
    //cout<<endl;
    //cout<<endl;
    idx.emplace_back(best_i1);
    idx.emplace_back(best_i2);
    return idx;
}
""")

True

In [3]:
#!xrdcp root://xrootd-cms.infn.it//store/user/apiccine/VBS/VBS_SSWW_TL_polarization_TuneCP5_13TeV-madgraph-pythia8/VBS_SSWW_TL_SM_2017/210420_124011/0000/tree_hadd_16.root .

[0B/0B][100%][==================================================][0B/s]  
Run: [ERROR] Server responded with an error: [3018] file exists (destination)



In [4]:
#!xrdcp root://xrootd-cms.infn.it//store/user/apiccine/VBS/VBS_SSWW_TL_polarization_TuneCP5_13TeV-madgraph-pythia8/VBS_SSWW_TL_SM_2017/210420_124011/0000/tree_hadd_17.root .

[421.6MB/421.6MB][100%][==================================================][4.133MB/s]  


In [5]:
#!xrdcp root://xrootd-cms.infn.it//store/user/apiccine/VBS/VBS_SSWW_TL_polarization_TuneCP5_13TeV-madgraph-pythia8/VBS_SSWW_TL_SM_2017/210420_124011/0000/tree_hadd_18.root .

[0B/0B][100%][==================================================][0B/s]  
Run: [ERROR] Server responded with an error: [3018] file exists (destination)



In [6]:
#!xrdcp root://xrootd-cms.infn.it//store/user/apiccine/VBS/VBS_SSWW_TT_polarization_TuneCP5_13TeV-madgraph-pythia8/VBS_SSWW_TT_SM_2017/210420_124141/0000/tree_hadd_9.root .

[84.35MB/84.35MB][100%][==================================================][12.05MB/s]  ^C


In [3]:
#!xrdcp root://xrootd-cms.infn.it//store/user/apiccine/VBS/VBS_SSWW_TT_polarization_TuneCP5_13TeV-madgraph-pythia8/VBS_SSWW_TT_SM_2017/210420_124141/0000/tree_hadd_10.root .

In [ ]:
!xrdcp root://xrootd-cms.infn.it//store/user/apiccine/VBS/VBS_SSWW_TT_polarization_TuneCP5_13TeV-madgraph-pythia8/VBS_SSWW_TT_SM_2017/210420_124141/0000/tree_hadd_11.root .

In [ ]:
#!xrdcp root://xrootd-cms.infn.it//store/user/apiccine/VBS/VBS_SSWW_LL_polarization_TuneCP5_13TeV-madgraph-pythia8/VBS_SSWW_LL_SM_2017/210420_123836/0000/tree_hadd_1.root .

In [4]:
chain = ["tree_hadd_16.root", "tree_hadd_18.root",  "tree_hadd_11.root"] #"tree_hadd_1.root"]

In [12]:
df = ROOT.RDataFrame("Events",chain)
#df = ROOT.RDataFrame("Events", "root://xrootd-cms.infn.it//store/user/ttedesch/VBS_PG/VBS_SSWW_LL_polarization_TuneCP5_13TeV-madgraph-pythia8/VBS_SSWW_LL_SM_2017/210908_151157/0000/tree_hadd_2.root",)
df_goodvertex = df.Filter("Flag_goodVertices && Flag_globalSuperTightHalo2016Filter && Flag_HBHENoiseFilter && Flag_HBHENoiseIsoFilter && Flag_EcalDeadCellTriggerPrimitiveFilter && Flag_BadPFMuonFilter && Flag_ecalBadCalibFilterV2", "Good vertex selection")
df_trigger = df_goodvertex.Filter("HLT_IsoMu27 || HLT_Mu50 || HLT_Ele35_WPTight_Gsf || HLT_Ele32_WPTight_Gsf_L1DoubleEG || HLT_Photon200", "Leptonic trigger")
df_atleast2Jets = df_trigger.Filter("nJet>=2", "At least two jets")
df_GenJets = df_atleast2Jets.Define("GenJet_idx", "SelectVBSQGenJet(GenPart_pdgId, GenPart_genPartIdxMother, GenPart_pt, GenPart_eta, GenJet_partonFlavour, GenJet_pt, GenJet_eta)")
df_GenJets_filtered = df_GenJets.Filter("GenJet_idx[0] != -9999", "Problem with genParts or GenJets")
df_atleast2GenJets = df_GenJets_filtered.Filter("atleast2Jets(GenJet_idx)","At least 2 GenJets")
df_GoodJets = df_atleast2GenJets.Define("GoodJets_idx", "GoodJets(Jet_jetId, Jet_eta, Jet_pt, Jet_puId)")
df_atleast2GoodJets = df_GoodJets.Filter("atleast2GoodJets(GoodJets_idx)", "At least two good jets")
df_VBSjets = df_atleast2GoodJets.Define("VBSJet_idx", "SelectVBSJets_invmass(Jet_pt, Jet_eta, Jet_phi, Jet_mass, GoodJets_idx)")
df_2VBSjets = df_VBSjets.Filter("VBSJet_idx[0] != VBSJet_idx[1]", "2 VBS jets").Define("mjj", "GetInvMass(Jet_pt, Jet_eta, Jet_phi, Jet_mass, VBSJet_idx)").Filter("mjj>500", "Good invariant mass").Define("Leadingjet_eta", "GetLeading(Jet_eta, VBSJet_idx)").Define("SubLeadingjet_eta", "GetSubLeading(Jet_eta, VBSJet_idx)").Filter("abs(Leadingjet_eta - SubLeadingjet_eta) > 2.5", "Good deltaEta")
df_defineGenMatching = df_2VBSjets.Define("leadjet_IsGenMatched", "IsGenMatched(GenJet_idx[0], Jet_genJetIdx[VBSJet_idx[0]])")\
                                         .Define("subleadjet_IsGenMatched", "IsGenMatched(GenJet_idx[1], Jet_genJetIdx[VBSJet_idx[1]])")\
                                         #.Filter("HLT_IsoMu27 || HLT_Mu50 || HLT_Ele35_WPTight_Gsf || HLT_Ele32_WPTight_Gsf_L1DoubleEG || HLT_Photon200", "Leptonic trigger")
df_Genmatched = df_defineGenMatching.Filter("leadjet_IsGenMatched == 1 && subleadjet_IsGenMatched == 1", "Genmatching")

In [13]:
allCutsReport = df.Report()
allCutsReport.Print()

Good vertex selection: pass=152169     all=152299     -- eff=99.91 % cumulative eff=99.91 %
Leptonic trigger: pass=152169     all=152169     -- eff=100.00 % cumulative eff=99.91 %
At least two jets: pass=152168     all=152169     -- eff=100.00 % cumulative eff=99.91 %
Problem with genParts or GenJets: pass=130331     all=152168     -- eff=85.65 % cumulative eff=85.58 %
At least 2 GenJets: pass=130331     all=130331     -- eff=100.00 % cumulative eff=85.58 %
At least two good jets: pass=127012     all=130331     -- eff=97.45 % cumulative eff=83.40 %
2 VBS jets: pass=102164     all=127012     -- eff=80.44 % cumulative eff=67.08 %
Good invariant mass: pass=87368      all=102164     -- eff=85.52 % cumulative eff=57.37 %
Good deltaEta: pass=87367      all=87368      -- eff=100.00 % cumulative eff=57.37 %
Genmatching: pass=60667      all=87367      -- eff=69.44 % cumulative eff=39.83 %


In [14]:
#df = ROOT.RDataFrame("Events", "VBS_SSWW_LL/AA101ED1-3427-E44D-9D6E-82569A5A589C.root ")
df = ROOT.RDataFrame("Events",chain)
#df = ROOT.RDataFrame("Events", "root://xrootd-cms.infn.it//store/user/ttedesch/VBS_PG/VBS_SSWW_LL_polarization_TuneCP5_13TeV-madgraph-pythia8/VBS_SSWW_LL_SM_2017/210908_151157/0000/tree_hadd_2.root",)
df_goodvertex = df.Filter("Flag_goodVertices && Flag_globalSuperTightHalo2016Filter && Flag_HBHENoiseFilter && Flag_HBHENoiseIsoFilter && Flag_EcalDeadCellTriggerPrimitiveFilter && Flag_BadPFMuonFilter && Flag_ecalBadCalibFilterV2", "Good vertex selection")
df_trigger = df_goodvertex.Filter("HLT_IsoMu27 || HLT_Mu50 || HLT_Ele35_WPTight_Gsf || HLT_Ele32_WPTight_Gsf_L1DoubleEG || HLT_Photon200", "Leptonic trigger")
df_atleast2Jets = df_trigger.Filter("nJet>=2", "At least two jets")
df_GenJets = df_atleast2Jets.Define("GenJet_idx", "SelectVBSQGenJet(GenPart_pdgId, GenPart_genPartIdxMother, GenPart_pt, GenPart_eta, GenJet_partonFlavour, GenJet_pt, GenJet_eta)")
df_GenJets_filtered = df_GenJets.Filter("GenJet_idx[0] != -9999", "Problem with genParts or GenJets")
df_atleast2GenJets = df_GenJets_filtered.Filter("atleast2Jets(GenJet_idx)","At least 2 GenJets")
df_GoodJets = df_atleast2GenJets.Define("GoodJets_idx", "GoodJets(Jet_jetId, Jet_eta, Jet_pt, Jet_puId)")
df_atleast2GoodJets = df_GoodJets.Filter("atleast2GoodJets(GoodJets_idx)", "At least two good jets")
df_VBSjets = df_atleast2GoodJets.Define("VBSJet_idx", "SelectVBSJets_tagger(Jet_pt, Jet_eta, Jet_phi, Jet_mass, Jet_area, Jet_chHEF, Jet_muEF, Jet_neEmEF, Jet_neHEF, Jet_jetId, Jet_nConstituents, Jet_nElectrons, Jet_nMuons, GoodJets_idx)")
df_2VBSjets = df_VBSjets.Filter("VBSJet_idx[0] != VBSJet_idx[1]", "2 VBS jets").Define("mjj", "GetInvMass(Jet_pt, Jet_eta, Jet_phi, Jet_mass, VBSJet_idx)").Filter("mjj>500", "Good invariant mass").Define("Leadingjet_eta", "GetLeading(Jet_eta, VBSJet_idx)").Define("SubLeadingjet_eta", "GetSubLeading(Jet_eta, VBSJet_idx)").Filter("abs(Leadingjet_eta - SubLeadingjet_eta) > 2.5", "Good deltaEta")
df_defineGenMatching = df_2VBSjets.Define("leadjet_IsGenMatched", "IsGenMatched(GenJet_idx[0], Jet_genJetIdx[VBSJet_idx[0]])")\
                                         .Define("subleadjet_IsGenMatched", "IsGenMatched(GenJet_idx[1], Jet_genJetIdx[VBSJet_idx[1]])")\
                                         #.Filter("HLT_IsoMu27 || HLT_Mu50 || HLT_Ele35_WPTight_Gsf || HLT_Ele32_WPTight_Gsf_L1DoubleEG || HLT_Photon200", "Leptonic trigger")
df_Genmatched = df_defineGenMatching.Filter("leadjet_IsGenMatched == 1 && subleadjet_IsGenMatched == 1", "Genmatching")

In [15]:
allCutsReport = df.Report()
allCutsReport.Print()

Good vertex selection: pass=152169     all=152299     -- eff=99.91 % cumulative eff=99.91 %
Leptonic trigger: pass=152169     all=152169     -- eff=100.00 % cumulative eff=99.91 %
At least two jets: pass=152168     all=152169     -- eff=100.00 % cumulative eff=99.91 %
Problem with genParts or GenJets: pass=130331     all=152168     -- eff=85.65 % cumulative eff=85.58 %
At least 2 GenJets: pass=130331     all=130331     -- eff=100.00 % cumulative eff=85.58 %
At least two good jets: pass=127012     all=130331     -- eff=97.45 % cumulative eff=83.40 %
2 VBS jets: pass=87368      all=127012     -- eff=68.79 % cumulative eff=57.37 %
Good invariant mass: pass=87368      all=87368      -- eff=100.00 % cumulative eff=57.37 %
Good deltaEta: pass=87367      all=87368      -- eff=100.00 % cumulative eff=57.37 %
Genmatching: pass=59657      all=87367      -- eff=68.28 % cumulative eff=39.17 %


In [9]:
#df = ROOT.RDataFrame("Events", "VBS_SSWW_LL/AA101ED1-3427-E44D-9D6E-82569A5A589C.root ")
#df = ROOT.RDataFrame("Events",["tree_hadd_17.root"])
df = ROOT.RDataFrame("Events",chain)
#df = ROOT.RDataFrame("Events", "root://xrootd-cms.infn.it//store/user/ttedesch/VBS_PG/VBS_SSWW_LL_polarization_TuneCP5_13TeV-madgraph-pythia8/VBS_SSWW_LL_SM_2017/210908_151157/0000/tree_hadd_2.root",)
df_goodvertex = df.Filter("Flag_goodVertices && Flag_globalSuperTightHalo2016Filter && Flag_HBHENoiseFilter && Flag_HBHENoiseIsoFilter && Flag_EcalDeadCellTriggerPrimitiveFilter && Flag_BadPFMuonFilter && Flag_ecalBadCalibFilterV2", "Good vertex selection")
df_trigger = df_goodvertex.Filter("HLT_IsoMu27 || HLT_Mu50 || HLT_Ele35_WPTight_Gsf || HLT_Ele32_WPTight_Gsf_L1DoubleEG || HLT_Photon200", "Leptonic trigger")
df_atleast2Jets = df_trigger.Filter("nJet>=2", "At least two jets")
df_GenJets = df_atleast2Jets.Define("GenJet_idx", "SelectVBSQGenJet(GenPart_pdgId, GenPart_genPartIdxMother, GenPart_pt, GenPart_eta, GenJet_partonFlavour, GenJet_pt, GenJet_eta)")
df_GenJets_filtered = df_GenJets.Filter("GenJet_idx[0] != -9999", "Problem with genParts or GenJets")
df_atleast2GenJets = df_GenJets_filtered.Filter("atleast2Jets(GenJet_idx)","At least 2 GenJets")
df_GoodJets = df_atleast2GenJets.Define("GoodJets_idx", "GoodJets(Jet_jetId, Jet_eta, Jet_pt, Jet_puId)")
df_atleast2GoodJets = df_GoodJets.Filter("atleast2GoodJets(GoodJets_idx)", "At least two good jets")
df_VBSjets = df_atleast2GoodJets.Define("VBSJet_idx", "SelectVBSJets_tagger_mjj(Jet_pt, Jet_eta, Jet_phi, Jet_mass, Jet_area, Jet_chHEF, Jet_muEF, Jet_neEmEF, Jet_neHEF, Jet_jetId, Jet_nConstituents, Jet_nElectrons, Jet_nMuons, GoodJets_idx)")
df_2VBSjets = df_VBSjets.Filter("VBSJet_idx[0] != VBSJet_idx[1]", "2 VBS jets").Define("mjj", "GetInvMass(Jet_pt, Jet_eta, Jet_phi, Jet_mass, VBSJet_idx)").Filter("mjj>500", "Good invariant mass").Define("Leadingjet_eta", "GetLeading(Jet_eta, VBSJet_idx)").Define("SubLeadingjet_eta", "GetSubLeading(Jet_eta, VBSJet_idx)").Filter("abs(Leadingjet_eta - SubLeadingjet_eta) > 2.5", "Good deltaEta")
df_defineGenMatching = df_2VBSjets.Define("leadjet_IsGenMatched", "IsGenMatched(GenJet_idx[0], Jet_genJetIdx[VBSJet_idx[0]])")\
                                         .Define("subleadjet_IsGenMatched", "IsGenMatched(GenJet_idx[1], Jet_genJetIdx[VBSJet_idx[1]])")\
                                         #.Filter("HLT_IsoMu27 || HLT_Mu50 || HLT_Ele35_WPTight_Gsf || HLT_Ele32_WPTight_Gsf_L1DoubleEG || HLT_Photon200", "Leptonic trigger")
df_Genmatched = df_defineGenMatching.Filter("leadjet_IsGenMatched == 1 && subleadjet_IsGenMatched == 1", "Genmatching")

In [10]:
allCutsReport = df.Report()
allCutsReport.Print()

Good vertex selection: pass=152169     all=152299     -- eff=99.91 % cumulative eff=99.91 %
Leptonic trigger: pass=152169     all=152169     -- eff=100.00 % cumulative eff=99.91 %
At least two jets: pass=152168     all=152169     -- eff=100.00 % cumulative eff=99.91 %
Problem with genParts or GenJets: pass=130331     all=152168     -- eff=85.65 % cumulative eff=85.58 %
At least 2 GenJets: pass=130331     all=130331     -- eff=100.00 % cumulative eff=85.58 %
At least two good jets: pass=127012     all=130331     -- eff=97.45 % cumulative eff=83.40 %
2 VBS jets: pass=87368      all=127012     -- eff=68.79 % cumulative eff=57.37 %
Good invariant mass: pass=87368      all=87368      -- eff=100.00 % cumulative eff=57.37 %
Good deltaEta: pass=87367      all=87368      -- eff=100.00 % cumulative eff=57.37 %
Genmatching: pass=62757      all=87367      -- eff=71.83 % cumulative eff=41.21 %


In [7]:
#df = ROOT.RDataFrame("Events", "VBS_SSWW_LL/AA101ED1-3427-E44D-9D6E-82569A5A589C.root ")
#df = ROOT.RDataFrame("Events",["tree_hadd_17.root"])
df = ROOT.RDataFrame("Events",chain)
#df = ROOT.RDataFrame("Events", "root://xrootd-cms.infn.it//store/user/ttedesch/VBS_PG/VBS_SSWW_LL_polarization_TuneCP5_13TeV-madgraph-pythia8/VBS_SSWW_LL_SM_2017/210908_151157/0000/tree_hadd_2.root",)
df_goodvertex = df.Filter("Flag_goodVertices && Flag_globalSuperTightHalo2016Filter && Flag_HBHENoiseFilter && Flag_HBHENoiseIsoFilter && Flag_EcalDeadCellTriggerPrimitiveFilter && Flag_BadPFMuonFilter && Flag_ecalBadCalibFilterV2", "Good vertex selection")
df_trigger = df_goodvertex.Filter("HLT_IsoMu27 || HLT_Mu50 || HLT_Ele35_WPTight_Gsf || HLT_Ele32_WPTight_Gsf_L1DoubleEG || HLT_Photon200", "Leptonic trigger")
df_atleast2Jets = df_trigger.Filter("nJet>=2", "At least two jets")
df_GenJets = df_atleast2Jets.Define("GenJet_idx", "SelectVBSQGenJet(GenPart_pdgId, GenPart_genPartIdxMother, GenPart_pt, GenPart_eta, GenJet_partonFlavour, GenJet_pt, GenJet_eta)")
df_GenJets_filtered = df_GenJets.Filter("GenJet_idx[0] != -9999", "Problem with genParts or GenJets")
df_atleast2GenJets = df_GenJets_filtered.Filter("atleast2Jets(GenJet_idx)","At least 2 GenJets")
df_GoodJets = df_atleast2GenJets.Define("GoodJets_idx", "GoodJets(Jet_jetId, Jet_eta, Jet_pt, Jet_puId)")
df_atleast2GoodJets = df_GoodJets.Filter("atleast2GoodJets(GoodJets_idx)", "At least two good jets")
df_VBSjets = df_atleast2GoodJets.Define("VBSJet_idx", "SelectVBSJets_tagger_mjj_absdeltaetajj(Jet_pt, Jet_eta, Jet_phi, Jet_mass, Jet_area, Jet_chHEF, Jet_muEF, Jet_neEmEF, Jet_neHEF, Jet_jetId, Jet_nConstituents, Jet_nElectrons, Jet_nMuons, GoodJets_idx)")
df_2VBSjets = df_VBSjets.Filter("VBSJet_idx[0] != VBSJet_idx[1]", "2 VBS jets").Define("mjj", "GetInvMass(Jet_pt, Jet_eta, Jet_phi, Jet_mass, VBSJet_idx)").Filter("mjj>500", "Good invariant mass").Define("Leadingjet_eta", "GetLeading(Jet_eta, VBSJet_idx)").Define("SubLeadingjet_eta", "GetSubLeading(Jet_eta, VBSJet_idx)").Filter("abs(Leadingjet_eta - SubLeadingjet_eta) > 2.5", "Good deltaEta")
df_defineGenMatching = df_2VBSjets.Define("leadjet_IsGenMatched", "IsGenMatched(GenJet_idx[0], Jet_genJetIdx[VBSJet_idx[0]])")\
                                         .Define("subleadjet_IsGenMatched", "IsGenMatched(GenJet_idx[1], Jet_genJetIdx[VBSJet_idx[1]])")\
                                         #.Filter("HLT_IsoMu27 || HLT_Mu50 || HLT_Ele35_WPTight_Gsf || HLT_Ele32_WPTight_Gsf_L1DoubleEG || HLT_Photon200", "Leptonic trigger")
df_Genmatched = df_defineGenMatching.Filter("leadjet_IsGenMatched == 1 && subleadjet_IsGenMatched == 1", "Genmatching")

In [8]:
allCutsReport = df.Report()
allCutsReport.Print()

Good vertex selection: pass=152169     all=152299     -- eff=99.91 % cumulative eff=99.91 %
Leptonic trigger: pass=152169     all=152169     -- eff=100.00 % cumulative eff=99.91 %
At least two jets: pass=152168     all=152169     -- eff=100.00 % cumulative eff=99.91 %
Problem with genParts or GenJets: pass=130331     all=152168     -- eff=85.65 % cumulative eff=85.58 %
At least 2 GenJets: pass=130331     all=130331     -- eff=100.00 % cumulative eff=85.58 %
At least two good jets: pass=127012     all=130331     -- eff=97.45 % cumulative eff=83.40 %
2 VBS jets: pass=87368      all=127012     -- eff=68.79 % cumulative eff=57.37 %
Good invariant mass: pass=87368      all=87368      -- eff=100.00 % cumulative eff=57.37 %
Good deltaEta: pass=87367      all=87368      -- eff=100.00 % cumulative eff=57.37 %
Genmatching: pass=62989      all=87367      -- eff=72.10 % cumulative eff=41.36 %


In [5]:
#df = ROOT.RDataFrame("Events", "VBS_SSWW_LL/AA101ED1-3427-E44D-9D6E-82569A5A589C.root ")
#df = ROOT.RDataFrame("Events",["tree_hadd_17.root"])
df = ROOT.RDataFrame("Events",chain)
#df = ROOT.RDataFrame("Events", "root://xrootd-cms.infn.it//store/user/ttedesch/VBS_PG/VBS_SSWW_LL_polarization_TuneCP5_13TeV-madgraph-pythia8/VBS_SSWW_LL_SM_2017/210908_151157/0000/tree_hadd_2.root",)
df_goodvertex = df.Filter("Flag_goodVertices && Flag_globalSuperTightHalo2016Filter && Flag_HBHENoiseFilter && Flag_HBHENoiseIsoFilter && Flag_EcalDeadCellTriggerPrimitiveFilter && Flag_BadPFMuonFilter && Flag_ecalBadCalibFilterV2", "Good vertex selection")
df_trigger = df_goodvertex.Filter("HLT_IsoMu27 || HLT_Mu50 || HLT_Ele35_WPTight_Gsf || HLT_Ele32_WPTight_Gsf_L1DoubleEG || HLT_Photon200", "Leptonic trigger")
df_atleast2Jets = df_trigger.Filter("nJet>=2", "At least two jets")
df_GenJets = df_atleast2Jets.Define("GenJet_idx", "SelectVBSQGenJet(GenPart_pdgId, GenPart_genPartIdxMother, GenPart_pt, GenPart_eta, GenJet_partonFlavour, GenJet_pt, GenJet_eta)")
df_GenJets_filtered = df_GenJets.Filter("GenJet_idx[0] != -9999", "Problem with genParts or GenJets")
df_atleast2GenJets = df_GenJets_filtered.Filter("atleast2Jets(GenJet_idx)","At least 2 GenJets")
df_GoodJets = df_atleast2GenJets.Define("GoodJets_idx", "GoodJets(Jet_jetId, Jet_eta, Jet_pt, Jet_puId)")
df_atleast2GoodJets = df_GoodJets.Filter("atleast2GoodJets(GoodJets_idx)", "At least two good jets")
df_VBSjets = df_atleast2GoodJets.Define("VBSJet_idx", "SelectVBSJets_tagger_mjj_absdeltaetajj_DNN(Jet_pt, Jet_eta, Jet_phi, Jet_mass, Jet_area, Jet_chHEF, Jet_muEF, Jet_neEmEF, Jet_neHEF, Jet_jetId, Jet_nConstituents, Jet_nElectrons, Jet_nMuons, GoodJets_idx)")
df_2VBSjets = df_VBSjets.Filter("VBSJet_idx[0] != VBSJet_idx[1]", "2 VBS jets").Define("mjj", "GetInvMass(Jet_pt, Jet_eta, Jet_phi, Jet_mass, VBSJet_idx)").Filter("mjj>500", "Good invariant mass").Define("Leadingjet_eta", "GetLeading(Jet_eta, VBSJet_idx)").Define("SubLeadingjet_eta", "GetSubLeading(Jet_eta, VBSJet_idx)").Filter("abs(Leadingjet_eta - SubLeadingjet_eta) > 2.5", "Good deltaEta")
df_defineGenMatching = df_2VBSjets.Define("leadjet_IsGenMatched", "IsGenMatched(GenJet_idx[0], Jet_genJetIdx[VBSJet_idx[0]])")\
                                         .Define("subleadjet_IsGenMatched", "IsGenMatched(GenJet_idx[1], Jet_genJetIdx[VBSJet_idx[1]])")\
                                         #.Filter("HLT_IsoMu27 || HLT_Mu50 || HLT_Ele35_WPTight_Gsf || HLT_Ele32_WPTight_Gsf_L1DoubleEG || HLT_Photon200", "Leptonic trigger")
df_Genmatched = df_defineGenMatching.Filter("leadjet_IsGenMatched == 1 && subleadjet_IsGenMatched == 1", "Genmatching")

Warning in <TClass::Init>: no dictionary for class edm::Hash<1> is available
Warning in <TClass::Init>: no dictionary for class edm::ProcessHistory is available
Warning in <TClass::Init>: no dictionary for class edm::ProcessConfiguration is available
Warning in <TClass::Init>: no dictionary for class edm::ParameterSetBlob is available
Warning in <TClass::Init>: no dictionary for class pair<edm::Hash<1>,edm::ParameterSetBlob> is available


In [6]:
allCutsReport = df.Report()
allCutsReport.Print()

Good vertex selection: pass=152169     all=152299     -- eff=99.91 % cumulative eff=99.91 %
Leptonic trigger: pass=152169     all=152169     -- eff=100.00 % cumulative eff=99.91 %
At least two jets: pass=152168     all=152169     -- eff=100.00 % cumulative eff=99.91 %
Problem with genParts or GenJets: pass=130331     all=152168     -- eff=85.65 % cumulative eff=85.58 %
At least 2 GenJets: pass=130331     all=130331     -- eff=100.00 % cumulative eff=85.58 %
At least two good jets: pass=127012     all=130331     -- eff=97.45 % cumulative eff=83.40 %
2 VBS jets: pass=87368      all=127012     -- eff=68.79 % cumulative eff=57.37 %
Good invariant mass: pass=87368      all=87368      -- eff=100.00 % cumulative eff=57.37 %
Good deltaEta: pass=87367      all=87368      -- eff=100.00 % cumulative eff=57.37 %
Genmatching: pass=62534      all=87367      -- eff=71.58 % cumulative eff=41.06 %


In [5]:
#df = ROOT.RDataFrame("Events", "VBS_SSWW_LL/AA101ED1-3427-E44D-9D6E-82569A5A589C.root ")
#df = ROOT.RDataFrame("Events",["tree_hadd_17.root"])
df = ROOT.RDataFrame("Events",chain)
#df = ROOT.RDataFrame("Events", "root://xrootd-cms.infn.it//store/user/ttedesch/VBS_PG/VBS_SSWW_LL_polarization_TuneCP5_13TeV-madgraph-pythia8/VBS_SSWW_LL_SM_2017/210908_151157/0000/tree_hadd_2.root",)
df_goodvertex = df.Filter("Flag_goodVertices && Flag_globalSuperTightHalo2016Filter && Flag_HBHENoiseFilter && Flag_HBHENoiseIsoFilter && Flag_EcalDeadCellTriggerPrimitiveFilter && Flag_BadPFMuonFilter && Flag_ecalBadCalibFilterV2", "Good vertex selection")
df_trigger = df_goodvertex.Filter("HLT_IsoMu27 || HLT_Mu50 || HLT_Ele35_WPTight_Gsf || HLT_Ele32_WPTight_Gsf_L1DoubleEG || HLT_Photon200", "Leptonic trigger")
df_atleast2Jets = df_trigger.Filter("nJet>=2", "At least two jets")
df_GenJets = df_atleast2Jets.Define("GenJet_idx", "SelectVBSQGenJet(GenPart_pdgId, GenPart_genPartIdxMother, GenPart_pt, GenPart_eta, GenJet_partonFlavour, GenJet_pt, GenJet_eta)")
df_GenJets_filtered = df_GenJets.Filter("GenJet_idx[0] != -9999", "Problem with genParts or GenJets")
df_atleast2GenJets = df_GenJets_filtered.Filter("atleast2Jets(GenJet_idx)","At least 2 GenJets")
df_GoodJets = df_atleast2GenJets.Define("GoodJets_idx", "GoodJets(Jet_jetId, Jet_eta, Jet_pt, Jet_puId)")
df_atleast2GoodJets = df_GoodJets.Filter("atleast2GoodJets(GoodJets_idx)", "At least two good jets")
df_VBSjets = df_atleast2GoodJets.Define("VBSJet_idx", "SelectVBSJets_tagger_mjj_absdeltaetajj_new(Jet_pt, Jet_eta, Jet_phi, Jet_mass, Jet_area, Jet_chHEF, Jet_muEF, Jet_neEmEF, Jet_neHEF, Jet_jetId, Jet_nConstituents, Jet_nElectrons, Jet_nMuons, GoodJets_idx)")
df_2VBSjets = df_VBSjets.Filter("VBSJet_idx[0] != VBSJet_idx[1]", "2 VBS jets").Define("mjj", "GetInvMass(Jet_pt, Jet_eta, Jet_phi, Jet_mass, VBSJet_idx)").Filter("mjj>500", "Good invariant mass").Define("Leadingjet_eta", "GetLeading(Jet_eta, VBSJet_idx)").Define("SubLeadingjet_eta", "GetSubLeading(Jet_eta, VBSJet_idx)").Filter("abs(Leadingjet_eta - SubLeadingjet_eta) > 2.5", "Good deltaEta")
df_defineGenMatching = df_2VBSjets.Define("leadjet_IsGenMatched", "IsGenMatched(GenJet_idx[0], Jet_genJetIdx[VBSJet_idx[0]])")\
                                         .Define("subleadjet_IsGenMatched", "IsGenMatched(GenJet_idx[1], Jet_genJetIdx[VBSJet_idx[1]])")\
                                         #.Filter("HLT_IsoMu27 || HLT_Mu50 || HLT_Ele35_WPTight_Gsf || HLT_Ele32_WPTight_Gsf_L1DoubleEG || HLT_Photon200", "Leptonic trigger")
df_Genmatched = df_defineGenMatching.Filter("leadjet_IsGenMatched == 1 && subleadjet_IsGenMatched == 1", "Genmatching")

Warning in <TClass::Init>: no dictionary for class edm::Hash<1> is available
Warning in <TClass::Init>: no dictionary for class edm::ProcessHistory is available
Warning in <TClass::Init>: no dictionary for class edm::ProcessConfiguration is available
Warning in <TClass::Init>: no dictionary for class edm::ParameterSetBlob is available
Warning in <TClass::Init>: no dictionary for class pair<edm::Hash<1>,edm::ParameterSetBlob> is available


In [6]:
allCutsReport = df.Report()
allCutsReport.Print()

Good vertex selection: pass=152169     all=152299     -- eff=99.91 % cumulative eff=99.91 %
Leptonic trigger: pass=152169     all=152169     -- eff=100.00 % cumulative eff=99.91 %
At least two jets: pass=152168     all=152169     -- eff=100.00 % cumulative eff=99.91 %
Problem with genParts or GenJets: pass=130331     all=152168     -- eff=85.65 % cumulative eff=85.58 %
At least 2 GenJets: pass=130331     all=130331     -- eff=100.00 % cumulative eff=85.58 %
At least two good jets: pass=127012     all=130331     -- eff=97.45 % cumulative eff=83.40 %
2 VBS jets: pass=87368      all=127012     -- eff=68.79 % cumulative eff=57.37 %
Good invariant mass: pass=87368      all=87368      -- eff=100.00 % cumulative eff=57.37 %
Good deltaEta: pass=87367      all=87368      -- eff=100.00 % cumulative eff=57.37 %
Genmatching: pass=63144      all=87367      -- eff=72.27 % cumulative eff=41.46 %


In [18]:
#df = ROOT.RDataFrame("Events", "VBS_SSWW_LL/AA101ED1-3427-E44D-9D6E-82569A5A589C.root ")
df = ROOT.RDataFrame("Events",chain)
#df = ROOT.RDataFrame("Events", "root://cms-xrd-global.cern.ch//store/user/ttedesch/VBS_PG/VBS_SSWW_LL_polarization_TuneCP5_13TeV-madgraph-pythia8/VBS_SSWW_LL_SM_2017/210908_151157/0000/tree_hadd_2.root",)
df_goodvertex = df.Filter("Flag_goodVertices && Flag_globalSuperTightHalo2016Filter && Flag_HBHENoiseFilter && Flag_HBHENoiseIsoFilter && Flag_EcalDeadCellTriggerPrimitiveFilter && Flag_BadPFMuonFilter && Flag_ecalBadCalibFilterV2", "Good vertex selection")
df_trigger = df_goodvertex.Filter("HLT_IsoMu27 || HLT_Mu50 || HLT_Ele35_WPTight_Gsf || HLT_Ele32_WPTight_Gsf_L1DoubleEG || HLT_Photon200", "Leptonic trigger")
df_atleast2Jets = df_trigger.Filter("nJet>=2", "At least two jets")
df_GenJets = df_atleast2Jets.Define("GenJet_idx", "SelectVBSQGenJet_FlavourMatching(GenPart_pdgId, GenPart_genPartIdxMother, GenPart_pt, GenPart_eta, GenJet_partonFlavour, GenJet_pt, GenJet_eta)")
df_GenJets_filtered = df_GenJets.Filter("GenJet_idx[0] != -9999", "Problem with genParts or GenJets")
df_atleast2GenJets = df_GenJets_filtered.Filter("atleast2Jets(GenJet_idx)","At least 2 GenJets")
df_GoodJets = df_atleast2GenJets.Define("GoodJets_idx", "GoodJets(Jet_jetId, Jet_eta, Jet_pt, Jet_puId)")
df_atleast2GoodJets = df_GoodJets.Filter("atleast2GoodJets(GoodJets_idx)", "At least two good jets")
df_VBSjets = df_atleast2GoodJets.Define("VBSJet_idx", "SelectVBSJets_invmass(Jet_pt, Jet_eta, Jet_phi, Jet_mass, GoodJets_idx)")
df_2VBSjets = df_VBSjets.Filter("VBSJet_idx[0] != VBSJet_idx[1]", "2 VBS jets").Define("mjj", "GetInvMass(Jet_pt, Jet_eta, Jet_phi, Jet_mass, VBSJet_idx)").Filter("mjj>500", "Good invariant mass").Define("Leadingjet_eta", "GetLeading(Jet_eta, VBSJet_idx)").Define("SubLeadingjet_eta", "GetSubLeading(Jet_eta, VBSJet_idx)").Filter("abs(Leadingjet_eta - SubLeadingjet_eta) > 2.5", "Good deltaEta")
df_defineGenMatching = df_2VBSjets.Define("leadjet_IsGenMatched", "IsGenMatched(GenJet_idx[0], Jet_genJetIdx[VBSJet_idx[0]])")\
                                         .Define("subleadjet_IsGenMatched", "IsGenMatched(GenJet_idx[1], Jet_genJetIdx[VBSJet_idx[1]])")\
                                         #.Filter("HLT_IsoMu27 || HLT_Mu50 || HLT_Ele35_WPTight_Gsf || HLT_Ele32_WPTight_Gsf_L1DoubleEG || HLT_Photon200", "Leptonic trigger")
df_Genmatched = df_defineGenMatching.Filter("leadjet_IsGenMatched == 1 && subleadjet_IsGenMatched == 1", "Genmatching")

In [19]:
allCutsReport = df.Report()
allCutsReport.Print()

Good vertex selection: pass=152169     all=152299     -- eff=99.91 % cumulative eff=99.91 %
Leptonic trigger: pass=152169     all=152169     -- eff=100.00 % cumulative eff=99.91 %
At least two jets: pass=152168     all=152169     -- eff=100.00 % cumulative eff=99.91 %
Problem with genParts or GenJets: pass=56820      all=152168     -- eff=37.34 % cumulative eff=37.31 %
At least 2 GenJets: pass=56820      all=56820      -- eff=100.00 % cumulative eff=37.31 %
At least two good jets: pass=54805      all=56820      -- eff=96.45 % cumulative eff=35.99 %
2 VBS jets: pass=42196      all=54805      -- eff=76.99 % cumulative eff=27.71 %
Good invariant mass: pass=34754      all=42196      -- eff=82.36 % cumulative eff=22.82 %
Good deltaEta: pass=34753      all=34754      -- eff=100.00 % cumulative eff=22.82 %
Genmatching: pass=25253      all=34753      -- eff=72.66 % cumulative eff=16.58 %


In [20]:
#df = ROOT.RDataFrame("Events", "VBS_SSWW_LL/AA101ED1-3427-E44D-9D6E-82569A5A589C.root ")
df = ROOT.RDataFrame("Events",chain)
#df = ROOT.RDataFrame("Events", "root://cms-xrd-global.cern.ch//store/user/ttedesch/VBS_PG/VBS_SSWW_LL_polarization_TuneCP5_13TeV-madgraph-pythia8/VBS_SSWW_LL_SM_2017/210908_151157/0000/tree_hadd_2.root",)
df_goodvertex = df.Filter("Flag_goodVertices && Flag_globalSuperTightHalo2016Filter && Flag_HBHENoiseFilter && Flag_HBHENoiseIsoFilter && Flag_EcalDeadCellTriggerPrimitiveFilter && Flag_BadPFMuonFilter && Flag_ecalBadCalibFilterV2", "Good vertex selection")
df_trigger = df_goodvertex.Filter("HLT_IsoMu27 || HLT_Mu50 || HLT_Ele35_WPTight_Gsf || HLT_Ele32_WPTight_Gsf_L1DoubleEG || HLT_Photon200", "Leptonic trigger")
df_atleast2Jets = df_trigger.Filter("nJet>=2", "At least two jets")
df_GenJets = df_atleast2Jets.Define("GenJet_idx", "SelectVBSQGenJet_FlavourMatching(GenPart_pdgId, GenPart_genPartIdxMother, GenPart_pt, GenPart_eta, GenJet_partonFlavour, GenJet_pt, GenJet_eta)")
df_GenJets_filtered = df_GenJets.Filter("GenJet_idx[0] != -9999", "Problem with genParts or GenJets")
df_atleast2GenJets = df_GenJets_filtered.Filter("atleast2Jets(GenJet_idx)","At least 2 GenJets")
df_GoodJets = df_atleast2GenJets.Define("GoodJets_idx", "GoodJets(Jet_jetId, Jet_eta, Jet_pt, Jet_puId)")
df_atleast2GoodJets = df_GoodJets.Filter("atleast2GoodJets(GoodJets_idx)", "At least two good jets")
df_VBSjets = df_atleast2GoodJets.Define("VBSJet_idx", "SelectVBSJets_tagger(Jet_pt, Jet_eta, Jet_phi, Jet_mass, Jet_area, Jet_chHEF, Jet_muEF, Jet_neEmEF, Jet_neHEF, Jet_jetId, Jet_nConstituents, Jet_nElectrons, Jet_nMuons, GoodJets_idx)")
df_2VBSjets = df_VBSjets.Filter("VBSJet_idx[0] != VBSJet_idx[1]", "2 VBS jets").Define("mjj", "GetInvMass(Jet_pt, Jet_eta, Jet_phi, Jet_mass, VBSJet_idx)").Filter("mjj>500", "Good invariant mass").Define("Leadingjet_eta", "GetLeading(Jet_eta, VBSJet_idx)").Define("SubLeadingjet_eta", "GetSubLeading(Jet_eta, VBSJet_idx)").Filter("abs(Leadingjet_eta - SubLeadingjet_eta) > 2.5", "Good deltaEta")
df_defineGenMatching = df_2VBSjets.Define("leadjet_IsGenMatched", "IsGenMatched(GenJet_idx[0], Jet_genJetIdx[VBSJet_idx[0]])")\
                                         .Define("subleadjet_IsGenMatched", "IsGenMatched(GenJet_idx[1], Jet_genJetIdx[VBSJet_idx[1]])")\
                                         #.Filter("HLT_IsoMu27 || HLT_Mu50 || HLT_Ele35_WPTight_Gsf || HLT_Ele32_WPTight_Gsf_L1DoubleEG || HLT_Photon200", "Leptonic trigger")
df_Genmatched = df_defineGenMatching.Filter("leadjet_IsGenMatched == 1 && subleadjet_IsGenMatched == 1", "Genmatching")

In [21]:
allCutsReport = df.Report()
allCutsReport.Print()

Good vertex selection: pass=152169     all=152299     -- eff=99.91 % cumulative eff=99.91 %
Leptonic trigger: pass=152169     all=152169     -- eff=100.00 % cumulative eff=99.91 %
At least two jets: pass=152168     all=152169     -- eff=100.00 % cumulative eff=99.91 %
Problem with genParts or GenJets: pass=56820      all=152168     -- eff=37.34 % cumulative eff=37.31 %
At least 2 GenJets: pass=56820      all=56820      -- eff=100.00 % cumulative eff=37.31 %
At least two good jets: pass=54805      all=56820      -- eff=96.45 % cumulative eff=35.99 %
2 VBS jets: pass=34754      all=54805      -- eff=63.41 % cumulative eff=22.82 %
Good invariant mass: pass=34754      all=34754      -- eff=100.00 % cumulative eff=22.82 %
Good deltaEta: pass=34753      all=34754      -- eff=100.00 % cumulative eff=22.82 %
Genmatching: pass=25923      all=34753      -- eff=74.59 % cumulative eff=17.02 %


In [22]:
#df = ROOT.RDataFrame("Events", "VBS_SSWW_LL/AA101ED1-3427-E44D-9D6E-82569A5A589C.root ")
df = ROOT.RDataFrame("Events",chain)
#df = ROOT.RDataFrame("Events", "root://cms-xrd-global.cern.ch//store/user/ttedesch/VBS_PG/VBS_SSWW_LL_polarization_TuneCP5_13TeV-madgraph-pythia8/VBS_SSWW_LL_SM_2017/210908_151157/0000/tree_hadd_2.root",)
df_goodvertex = df.Filter("Flag_goodVertices && Flag_globalSuperTightHalo2016Filter && Flag_HBHENoiseFilter && Flag_HBHENoiseIsoFilter && Flag_EcalDeadCellTriggerPrimitiveFilter && Flag_BadPFMuonFilter && Flag_ecalBadCalibFilterV2", "Good vertex selection")
df_trigger = df_goodvertex.Filter("HLT_IsoMu27 || HLT_Mu50 || HLT_Ele35_WPTight_Gsf || HLT_Ele32_WPTight_Gsf_L1DoubleEG || HLT_Photon200", "Leptonic trigger")
df_atleast2Jets = df_trigger.Filter("nJet>=2", "At least two jets")
df_GenJets = df_atleast2Jets.Define("GenJet_idx", "SelectVBSQGenJet_FlavourMatching(GenPart_pdgId, GenPart_genPartIdxMother, GenPart_pt, GenPart_eta, GenJet_partonFlavour, GenJet_pt, GenJet_eta)")
df_GenJets_filtered = df_GenJets.Filter("GenJet_idx[0] != -9999", "Problem with genParts or GenJets")
df_atleast2GenJets = df_GenJets_filtered.Filter("atleast2Jets(GenJet_idx)","At least 2 GenJets")
df_GoodJets = df_atleast2GenJets.Define("GoodJets_idx", "GoodJets(Jet_jetId, Jet_eta, Jet_pt, Jet_puId)")
df_atleast2GoodJets = df_GoodJets.Filter("atleast2GoodJets(GoodJets_idx)", "At least two good jets")
df_VBSjets = df_atleast2GoodJets.Define("VBSJet_idx", "SelectVBSJets_tagger_mjj(Jet_pt, Jet_eta, Jet_phi, Jet_mass, Jet_area, Jet_chHEF, Jet_muEF, Jet_neEmEF, Jet_neHEF, Jet_jetId, Jet_nConstituents, Jet_nElectrons, Jet_nMuons, GoodJets_idx)")
df_2VBSjets = df_VBSjets.Filter("VBSJet_idx[0] != VBSJet_idx[1]", "2 VBS jets").Define("mjj", "GetInvMass(Jet_pt, Jet_eta, Jet_phi, Jet_mass, VBSJet_idx)").Filter("mjj>500", "Good invariant mass").Define("Leadingjet_eta", "GetLeading(Jet_eta, VBSJet_idx)").Define("SubLeadingjet_eta", "GetSubLeading(Jet_eta, VBSJet_idx)").Filter("abs(Leadingjet_eta - SubLeadingjet_eta) > 2.5", "Good deltaEta")
df_defineGenMatching = df_2VBSjets.Define("leadjet_IsGenMatched", "IsGenMatched(GenJet_idx[0], Jet_genJetIdx[VBSJet_idx[0]])")\
                                         .Define("subleadjet_IsGenMatched", "IsGenMatched(GenJet_idx[1], Jet_genJetIdx[VBSJet_idx[1]])")\
                                         #.Filter("HLT_IsoMu27 || HLT_Mu50 || HLT_Ele35_WPTight_Gsf || HLT_Ele32_WPTight_Gsf_L1DoubleEG || HLT_Photon200", "Leptonic trigger")
df_Genmatched = df_defineGenMatching.Filter("leadjet_IsGenMatched == 1 && subleadjet_IsGenMatched == 1", "Genmatching")

In [23]:
allCutsReport = df.Report()
allCutsReport.Print()

Good vertex selection: pass=152169     all=152299     -- eff=99.91 % cumulative eff=99.91 %
Leptonic trigger: pass=152169     all=152169     -- eff=100.00 % cumulative eff=99.91 %
At least two jets: pass=152168     all=152169     -- eff=100.00 % cumulative eff=99.91 %
Problem with genParts or GenJets: pass=56820      all=152168     -- eff=37.34 % cumulative eff=37.31 %
At least 2 GenJets: pass=56820      all=56820      -- eff=100.00 % cumulative eff=37.31 %
At least two good jets: pass=54805      all=56820      -- eff=96.45 % cumulative eff=35.99 %
2 VBS jets: pass=34754      all=54805      -- eff=63.41 % cumulative eff=22.82 %
Good invariant mass: pass=34754      all=34754      -- eff=100.00 % cumulative eff=22.82 %
Good deltaEta: pass=34753      all=34754      -- eff=100.00 % cumulative eff=22.82 %
Genmatching: pass=26542      all=34753      -- eff=76.37 % cumulative eff=17.43 %


In [4]:
#df = ROOT.RDataFrame("Events", "VBS_SSWW_LL/AA101ED1-3427-E44D-9D6E-82569A5A589C.root ")
df = ROOT.RDataFrame("Events",chain)
#df = ROOT.RDataFrame("Events", "root://cms-xrd-global.cern.ch//store/user/ttedesch/VBS_PG/VBS_SSWW_LL_polarization_TuneCP5_13TeV-madgraph-pythia8/VBS_SSWW_LL_SM_2017/210908_151157/0000/tree_hadd_2.root",)
df_goodvertex = df.Filter("Flag_goodVertices && Flag_globalSuperTightHalo2016Filter && Flag_HBHENoiseFilter && Flag_HBHENoiseIsoFilter && Flag_EcalDeadCellTriggerPrimitiveFilter && Flag_BadPFMuonFilter && Flag_ecalBadCalibFilterV2", "Good vertex selection")
df_trigger = df_goodvertex.Filter("HLT_IsoMu27 || HLT_Mu50 || HLT_Ele35_WPTight_Gsf || HLT_Ele32_WPTight_Gsf_L1DoubleEG || HLT_Photon200", "Leptonic trigger")
df_atleast2Jets = df_trigger.Filter("nJet>=2", "At least two jets")
df_GenJets = df_atleast2Jets.Define("GenJet_idx", "SelectVBSQGenJet_FlavourMatching(GenPart_pdgId, GenPart_genPartIdxMother, GenPart_pt, GenPart_eta, GenJet_partonFlavour, GenJet_pt, GenJet_eta)")
df_GenJets_filtered = df_GenJets.Filter("GenJet_idx[0] != -9999", "Problem with genParts or GenJets")
df_atleast2GenJets = df_GenJets_filtered.Filter("atleast2Jets(GenJet_idx)","At least 2 GenJets")
df_GoodJets = df_atleast2GenJets.Define("GoodJets_idx", "GoodJets(Jet_jetId, Jet_eta, Jet_pt, Jet_puId)")
df_atleast2GoodJets = df_GoodJets.Filter("atleast2GoodJets(GoodJets_idx)", "At least two good jets")
df_VBSjets = df_atleast2GoodJets.Define("VBSJet_idx", "SelectVBSJets_tagger_mjj_absdeltaetajj_flavourmatching(Jet_pt, Jet_eta, Jet_phi, Jet_mass, Jet_area, Jet_chHEF, Jet_muEF, Jet_neEmEF, Jet_neHEF, Jet_jetId, Jet_nConstituents, Jet_nElectrons, Jet_nMuons, GoodJets_idx)")
df_2VBSjets = df_VBSjets.Filter("VBSJet_idx[0] != VBSJet_idx[1]", "2 VBS jets").Define("mjj", "GetInvMass(Jet_pt, Jet_eta, Jet_phi, Jet_mass, VBSJet_idx)").Filter("mjj>500", "Good invariant mass").Define("Leadingjet_eta", "GetLeading(Jet_eta, VBSJet_idx)").Define("SubLeadingjet_eta", "GetSubLeading(Jet_eta, VBSJet_idx)").Filter("abs(Leadingjet_eta - SubLeadingjet_eta) > 2.5", "Good deltaEta")
df_defineGenMatching = df_2VBSjets.Define("leadjet_IsGenMatched", "IsGenMatched(GenJet_idx[0], Jet_genJetIdx[VBSJet_idx[0]])")\
                                         .Define("subleadjet_IsGenMatched", "IsGenMatched(GenJet_idx[1], Jet_genJetIdx[VBSJet_idx[1]])")\
                                         #.Filter("HLT_IsoMu27 || HLT_Mu50 || HLT_Ele35_WPTight_Gsf || HLT_Ele32_WPTight_Gsf_L1DoubleEG || HLT_Photon200", "Leptonic trigger")
df_Genmatched = df_defineGenMatching.Filter("leadjet_IsGenMatched == 1 && subleadjet_IsGenMatched == 1", "Genmatching")

Warning in <TClass::Init>: no dictionary for class edm::Hash<1> is available
Warning in <TClass::Init>: no dictionary for class edm::ProcessHistory is available
Warning in <TClass::Init>: no dictionary for class edm::ProcessConfiguration is available
Warning in <TClass::Init>: no dictionary for class edm::ParameterSetBlob is available
Warning in <TClass::Init>: no dictionary for class pair<edm::Hash<1>,edm::ParameterSetBlob> is available


In [5]:
allCutsReport = df.Report()
allCutsReport.Print()

Good vertex selection: pass=152169     all=152299     -- eff=99.91 % cumulative eff=99.91 %
Leptonic trigger: pass=152169     all=152169     -- eff=100.00 % cumulative eff=99.91 %
At least two jets: pass=152168     all=152169     -- eff=100.00 % cumulative eff=99.91 %
Problem with genParts or GenJets: pass=56820      all=152168     -- eff=37.34 % cumulative eff=37.31 %
At least 2 GenJets: pass=56820      all=56820      -- eff=100.00 % cumulative eff=37.31 %
At least two good jets: pass=54805      all=56820      -- eff=96.45 % cumulative eff=35.99 %
2 VBS jets: pass=34754      all=54805      -- eff=63.41 % cumulative eff=22.82 %
Good invariant mass: pass=34754      all=34754      -- eff=100.00 % cumulative eff=22.82 %
Good deltaEta: pass=34753      all=34754      -- eff=100.00 % cumulative eff=22.82 %
Genmatching: pass=26767      all=34753      -- eff=77.02 % cumulative eff=17.58 %


In [4]:
#df = ROOT.RDataFrame("Events", "VBS_SSWW_LL/AA101ED1-3427-E44D-9D6E-82569A5A589C.root ")
df = ROOT.RDataFrame("Events",chain)
#df = ROOT.RDataFrame("Events", "root://cms-xrd-global.cern.ch//store/user/ttedesch/VBS_PG/VBS_SSWW_LL_polarization_TuneCP5_13TeV-madgraph-pythia8/VBS_SSWW_LL_SM_2017/210908_151157/0000/tree_hadd_2.root",)
df_goodvertex = df.Filter("Flag_goodVertices && Flag_globalSuperTightHalo2016Filter && Flag_HBHENoiseFilter && Flag_HBHENoiseIsoFilter && Flag_EcalDeadCellTriggerPrimitiveFilter && Flag_BadPFMuonFilter && Flag_ecalBadCalibFilterV2", "Good vertex selection")
df_trigger = df_goodvertex.Filter("HLT_IsoMu27 || HLT_Mu50 || HLT_Ele35_WPTight_Gsf || HLT_Ele32_WPTight_Gsf_L1DoubleEG || HLT_Photon200", "Leptonic trigger")
df_atleast2Jets = df_trigger.Filter("nJet>=2", "At least two jets")
df_GenJets = df_atleast2Jets.Define("GenJet_idx", "SelectVBSQGenJet_FlavourMatching(GenPart_pdgId, GenPart_genPartIdxMother, GenPart_pt, GenPart_eta, GenJet_partonFlavour, GenJet_pt, GenJet_eta)")
df_GenJets_filtered = df_GenJets.Filter("GenJet_idx[0] != -9999", "Problem with genParts or GenJets")
df_atleast2GenJets = df_GenJets_filtered.Filter("atleast2Jets(GenJet_idx)","At least 2 GenJets")
df_GoodJets = df_atleast2GenJets.Define("GoodJets_idx", "GoodJets(Jet_jetId, Jet_eta, Jet_pt, Jet_puId)")
df_atleast2GoodJets = df_GoodJets.Filter("atleast2GoodJets(GoodJets_idx)", "At least two good jets")
df_VBSjets = df_atleast2GoodJets.Define("VBSJet_idx", "SelectVBSJets_tagger_mjj_absdeltaetajj_flavourmatching_clean_fixed(Jet_pt, Jet_eta, Jet_phi, Jet_mass, Jet_area, Jet_chHEF, Jet_muEF, Jet_neEmEF, Jet_neHEF, Jet_jetId, Jet_nConstituents, Jet_nElectrons, Jet_nMuons, GoodJets_idx)")
df_2VBSjets = df_VBSjets.Filter("VBSJet_idx[0] != VBSJet_idx[1]", "2 VBS jets").Define("mjj", "GetInvMass(Jet_pt, Jet_eta, Jet_phi, Jet_mass, VBSJet_idx)").Filter("mjj>500", "Good invariant mass").Define("Leadingjet_eta", "GetLeading(Jet_eta, VBSJet_idx)").Define("SubLeadingjet_eta", "GetSubLeading(Jet_eta, VBSJet_idx)").Filter("abs(Leadingjet_eta - SubLeadingjet_eta) > 2.5", "Good deltaEta")
df_defineGenMatching = df_2VBSjets.Define("leadjet_IsGenMatched", "IsGenMatched(GenJet_idx[0], Jet_genJetIdx[VBSJet_idx[0]])")\
                                         .Define("subleadjet_IsGenMatched", "IsGenMatched(GenJet_idx[1], Jet_genJetIdx[VBSJet_idx[1]])")\
                                         #.Filter("HLT_IsoMu27 || HLT_Mu50 || HLT_Ele35_WPTight_Gsf || HLT_Ele32_WPTight_Gsf_L1DoubleEG || HLT_Photon200", "Leptonic trigger")
df_Genmatched = df_defineGenMatching.Filter("leadjet_IsGenMatched == 1 && subleadjet_IsGenMatched == 1", "Genmatching")

Warning in <TClass::Init>: no dictionary for class edm::Hash<1> is available
Warning in <TClass::Init>: no dictionary for class edm::ProcessHistory is available
Warning in <TClass::Init>: no dictionary for class edm::ProcessConfiguration is available
Warning in <TClass::Init>: no dictionary for class edm::ParameterSetBlob is available
Warning in <TClass::Init>: no dictionary for class pair<edm::Hash<1>,edm::ParameterSetBlob> is available


In [5]:
allCutsReport = df.Report()
allCutsReport.Print()

Good vertex selection: pass=152169     all=152299     -- eff=99.91 % cumulative eff=99.91 %
Leptonic trigger: pass=152169     all=152169     -- eff=100.00 % cumulative eff=99.91 %
At least two jets: pass=152168     all=152169     -- eff=100.00 % cumulative eff=99.91 %
Problem with genParts or GenJets: pass=56820      all=152168     -- eff=37.34 % cumulative eff=37.31 %
At least 2 GenJets: pass=56820      all=56820      -- eff=100.00 % cumulative eff=37.31 %
At least two good jets: pass=54805      all=56820      -- eff=96.45 % cumulative eff=35.99 %
2 VBS jets: pass=34754      all=54805      -- eff=63.41 % cumulative eff=22.82 %
Good invariant mass: pass=34754      all=34754      -- eff=100.00 % cumulative eff=22.82 %
Good deltaEta: pass=34753      all=34754      -- eff=100.00 % cumulative eff=22.82 %
Genmatching: pass=26751      all=34753      -- eff=76.97 % cumulative eff=17.56 %


In [5]:
#df = ROOT.RDataFrame("Events", "VBS_SSWW_LL/AA101ED1-3427-E44D-9D6E-82569A5A589C.root ")
df = ROOT.RDataFrame("Events",chain)
#df = ROOT.RDataFrame("Events", "root://cms-xrd-global.cern.ch//store/user/ttedesch/VBS_PG/VBS_SSWW_LL_polarization_TuneCP5_13TeV-madgraph-pythia8/VBS_SSWW_LL_SM_2017/210908_151157/0000/tree_hadd_2.root",)
df_goodvertex = df.Filter("Flag_goodVertices && Flag_globalSuperTightHalo2016Filter && Flag_HBHENoiseFilter && Flag_HBHENoiseIsoFilter && Flag_EcalDeadCellTriggerPrimitiveFilter && Flag_BadPFMuonFilter && Flag_ecalBadCalibFilterV2", "Good vertex selection")
df_trigger = df_goodvertex.Filter("HLT_IsoMu27 || HLT_Mu50 || HLT_Ele35_WPTight_Gsf || HLT_Ele32_WPTight_Gsf_L1DoubleEG || HLT_Photon200", "Leptonic trigger")
df_atleast2Jets = df_trigger.Filter("nJet>=2", "At least two jets")
df_GenJets = df_atleast2Jets.Define("GenJet_idx", "SelectVBSQGenJet_FlavourMatching(GenPart_pdgId, GenPart_genPartIdxMother, GenPart_pt, GenPart_eta, GenJet_partonFlavour, GenJet_pt, GenJet_eta)")
df_GenJets_filtered = df_GenJets.Filter("GenJet_idx[0] != -9999", "Problem with genParts or GenJets")
df_atleast2GenJets = df_GenJets_filtered.Filter("atleast2Jets(GenJet_idx)","At least 2 GenJets")
df_GoodJets = df_atleast2GenJets.Define("GoodJets_idx", "GoodJets(Jet_jetId, Jet_eta, Jet_pt, Jet_puId)")
df_atleast2GoodJets = df_GoodJets.Filter("atleast2GoodJets(GoodJets_idx)", "At least two good jets")
df_VBSjets = df_atleast2GoodJets.Define("VBSJet_idx", "SelectVBSJets_tagger_mjj_absdeltaetajj_flavourmatching_clean_fixed_central(Jet_pt, Jet_eta, Jet_phi, Jet_mass, Jet_area, Jet_chHEF, Jet_muEF, Jet_neEmEF, Jet_neHEF, Jet_jetId, Jet_nConstituents, Jet_nElectrons, Jet_nMuons, GoodJets_idx, Electron_pt, Electron_eta, Electron_phi, Muon_pt, Muon_eta, Muon_phi)")
df_2VBSjets = df_VBSjets.Filter("VBSJet_idx[0] != VBSJet_idx[1]", "2 VBS jets").Define("mjj", "GetInvMass(Jet_pt, Jet_eta, Jet_phi, Jet_mass, VBSJet_idx)").Filter("mjj>500", "Good invariant mass").Define("Leadingjet_eta", "GetLeading(Jet_eta, VBSJet_idx)").Define("SubLeadingjet_eta", "GetSubLeading(Jet_eta, VBSJet_idx)").Filter("abs(Leadingjet_eta - SubLeadingjet_eta) > 2.5", "Good deltaEta")
df_defineGenMatching = df_2VBSjets.Define("leadjet_IsGenMatched", "IsGenMatched(GenJet_idx[0], Jet_genJetIdx[VBSJet_idx[0]])")\
                                         .Define("subleadjet_IsGenMatched", "IsGenMatched(GenJet_idx[1], Jet_genJetIdx[VBSJet_idx[1]])")\
                                         #.Filter("HLT_IsoMu27 || HLT_Mu50 || HLT_Ele35_WPTight_Gsf || HLT_Ele32_WPTight_Gsf_L1DoubleEG || HLT_Photon200", "Leptonic trigger")
df_Genmatched = df_defineGenMatching.Filter("leadjet_IsGenMatched == 1 && subleadjet_IsGenMatched == 1", "Genmatching")

Warning in <TClass::Init>: no dictionary for class edm::Hash<1> is available
Warning in <TClass::Init>: no dictionary for class edm::ProcessHistory is available
Warning in <TClass::Init>: no dictionary for class edm::ProcessConfiguration is available
Warning in <TClass::Init>: no dictionary for class edm::ParameterSetBlob is available
Warning in <TClass::Init>: no dictionary for class pair<edm::Hash<1>,edm::ParameterSetBlob> is available


In [6]:
allCutsReport = df.Report()
allCutsReport.Print()

Good vertex selection: pass=152169     all=152299     -- eff=99.91 % cumulative eff=99.91 %
Leptonic trigger: pass=152169     all=152169     -- eff=100.00 % cumulative eff=99.91 %
At least two jets: pass=152168     all=152169     -- eff=100.00 % cumulative eff=99.91 %
Problem with genParts or GenJets: pass=56820      all=152168     -- eff=37.34 % cumulative eff=37.31 %
At least 2 GenJets: pass=56820      all=56820      -- eff=100.00 % cumulative eff=37.31 %
At least two good jets: pass=54805      all=56820      -- eff=96.45 % cumulative eff=35.99 %
2 VBS jets: pass=34754      all=54805      -- eff=63.41 % cumulative eff=22.82 %
Good invariant mass: pass=34754      all=34754      -- eff=100.00 % cumulative eff=22.82 %
Good deltaEta: pass=34753      all=34754      -- eff=100.00 % cumulative eff=22.82 %
Genmatching: pass=26803      all=34753      -- eff=77.12 % cumulative eff=17.60 %
